# Examples

The following examples, which are part of the documentation, demonstrate some of the situations in which you would want to use superintendent. Choose an example, and if you're finding you'd like to do more - read the [documentation](http://www.janfreyberg.com/superintendent/).

1. [Labelling text (UK newspaper headlines)](docs/examples/labelling-text.ipynb)
2. [Labelling images](docs/examples/labelling-images.ipynb)
3. [Labelling images with active learning](docs/examples/labelling-images-actively.ipynb)
4. [Applying different preprocessing for displaying and for modelling](docs/examples/preprocessing-data.ipynb)
